<a href="https://colab.research.google.com/github/roque-alfaro/taller-eiv-2026/blob/main/5_Ejercicio_ETL_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio:

Utilizando los datos de interconsultas del SSMN entre 2019 y 2023, cree un flujo de Extracción, Transformación y Carga para responder: ¿Cuál fue la demanda de interconsultas de Gastroenterología y Coloproctología por centro entre 2019 y 2024?


# Configurar área de trabajo y cargar datos
Usaremos las librerías de [Plotly](https://plotly.com/graphing-libraries/) para construir gráficos interactivos 📊✨.

In [1]:
# importar librerías
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# definir carpetas de trabajo
datos_crudos_interconsultas = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/datos_crudos/Solicitud%20Informaci%C3%B3n%20LT-AO008T00001801.parquet'
path_dw = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/datos_multidimensionales/Data_Warehouse_Dims.xlsx'

# Extraer

Explique a continuación qué es lo que hace este bloque de código:

In [4]:
sic_raw = pd.read_parquet(datos_crudos_interconsultas)
print(sic_raw.info())
sic_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398520 entries, 0 to 398519
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   SEXO        398520 non-null  int64         
 1   TIPO_PREST  398520 non-null  int64         
 2   PRESTA_MIN  398520 non-null  object        
 3   F_ENTRADA   398520 non-null  datetime64[ns]
 4   ESTAB_DEST  398520 non-null  int64         
 5   EDAD        398520 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 18.2+ MB
None


,SEXO,TIPO_PREST,PRESTA_MIN,F_ENTRADA,ESTAB_DEST,EDAD
0,1,1,07-041,2019-01-11,109101,3
1,2,1,09-011,2019-01-08,109100,35
2,2,1,09-011,2019-01-08,109100,35
3,2,1,07-044,2019-01-11,109309,27
4,2,1,07-066,2019-01-09,109100,27
...,...,...,...,...,...,...
398515,2,1,09-003,2023-12-30,109100,39
398516,2,1,09-011,2023-12-30,109100,74
398517,1,1,07-054,2023-12-31,109200,70
398518,2,1,07-018,2023-12-31,109100,68


# Transformar SIC

In [10]:
import pandas as pd

# Asegurar fecha
sic = sic_raw.copy()
sic["F_ENTRADA"] = pd.to_datetime(sic["F_ENTRADA"], errors="coerce")

# Año/Mes
sic["Año"] = sic["F_ENTRADA"].dt.year
sic["Mes"] = sic["F_ENTRADA"].dt.month

# --------- Helper para encontrar nombres reales de columnas ----------
def pick_col(df, candidates, label):
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(
        f"No encontré la columna para '{label}'. Probé: {candidates}. "
        f"Columnas disponibles: {list(df.columns)}"
    )

sexo_col = pick_col(sic, ["SEXO", "Sexo"], "sexo")
prest_col = pick_col(sic, ["PRESTA_MIN", "PRESTACION", "Prestacion", "COD_PRESTACION", "Prestación", "PRESTACION_COD"], "prestación")
estab_col = pick_col(sic, ["ESTAB_DEST", "ESTAB_DESTINO", "Centro_destino", "ESTABLECIMIENTO_DESTINO", "ID_ESTAB_DESTINO"], "establecimiento destino")
edad_col = pick_col(sic, ["EDAD", "Edad"], "edad")

# --------- ESTA ES TU LÍNEA COMPLETADA (pero robusta) ----------
sic = (
    sic[[sexo_col, prest_col, estab_col, edad_col, "Año", "Mes"]]
      .value_counts(sort=False)
      .reset_index(name="Interconsultas")
      .rename(columns={
          sexo_col: "SEXO",
          prest_col: "PRESTA_MIN",
          estab_col: "ESTAB_DEST",
          edad_col: "EDAD"
      })
)

sic.head()

,SEXO,PRESTA_MIN,ESTAB_DEST,EDAD,Año,Mes,Interconsultas
0,1,07-002,109101,0,2020,1,3
1,1,07-002,109101,0,2020,2,1
2,1,07-002,109101,0,2020,4,1
3,1,07-002,109101,0,2020,6,1
4,1,07-002,109101,0,2020,7,1


# Crear cubo

In [11]:
# Cargar dimensiones

# 1. Centros: Leer, seleccionar 3 columnas (Código, especialidad y nombre de la comuna) y renombrar campo Comuna
centros = pd.read_excel(path_dw, sheet_name='centros')[['Codigo','Nombre','Nombre Comuna']].rename(columns={'Nombre':'Centro','Nombre Comuna':'Comuna'})
# 2. Sexo: Leer hoja completa
sexo = pd.read_excel(path_dw, sheet_name='sexo')
# 3. Edad: Leer hoja completa
edad = pd.read_excel(path_dw, sheet_name='edad')
# 4. Especialidad: Leer y renombrar Nombre por Especialidad
especialidad = pd.read_excel(path_dw, sheet_name='prestacion').rename(columns={'Nombre':'Especialidad'})

¿Qué data frames cumplen las siguientes funciones en esta base de datos?
*   **Fact**= _________________
*   **Dimensión 1**= _________________
*   **Dimensión 2**= _________________
*   **Dimensión 3**= _________________
*   **Dimensión 4**= _________________



In [12]:
#Generamos el cubo
cubo = (sic.rename(columns={'count':'Interconsultas'}). # 1. Renombrar 'count' a Interconsultas
        # 2. Merge con sexo
        merge(sexo, left_on='SEXO', right_on='Codigo', how='left').
        # 3. Merge con centros
        merge(centros, left_on='ESTAB_DEST', right_on='Codigo', how='left').
        # 4. Merge con edad
        merge(edad, left_on='EDAD', right_on='Edad', how='left').
        # 5. Merge con especialidad
        merge(especialidad, left_on='PRESTA_MIN', right_on='Codigo', how='left'))

# 7. Selección final de columnas
cubo[['Año','Mes','Especialidad','Centro','Comuna','Sexo','Edad','RangoEdad5','RangoEdad10','Interconsultas']]

,Año,Mes,Especialidad,Centro,Comuna,Sexo,Edad,RangoEdad5,RangoEdad10,Interconsultas
0,2020,1,ANESTESIOLOGÍA,Hospital Clínico de Niños Dr. Roberto del Río ...,Independencia,Hombre,0.0,"[0, 5)","[0, 10)",3
1,2020,2,ANESTESIOLOGÍA,Hospital Clínico de Niños Dr. Roberto del Río ...,Independencia,Hombre,0.0,"[0, 5)","[0, 10)",1
2,2020,4,ANESTESIOLOGÍA,Hospital Clínico de Niños Dr. Roberto del Río ...,Independencia,Hombre,0.0,"[0, 5)","[0, 10)",1
3,2020,6,ANESTESIOLOGÍA,Hospital Clínico de Niños Dr. Roberto del Río ...,Independencia,Hombre,0.0,"[0, 5)","[0, 10)",1
4,2020,7,ANESTESIOLOGÍA,Hospital Clínico de Niños Dr. Roberto del Río ...,Independencia,Hombre,0.0,"[0, 5)","[0, 10)",1
...,...,...,...,...,...,...,...,...,...,...
188806,2021,11,CIRUGÍA DIGESTIVA,"Complejo Hospitalario San José (Santiago, Inde...",Independencia,Desconocido,34.0,"[30, 35)","[30, 40)",1
188807,2021,8,GINECOLOGÍA,"Complejo Hospitalario San José (Santiago, Inde...",Independencia,Desconocido,33.0,"[30, 35)","[30, 40)",1
188808,2021,12,OBSTETRICIA,"Complejo Hospitalario San José (Santiago, Inde...",Independencia,Desconocido,22.0,"[20, 25)","[20, 30)",1
188809,2020,12,CIRUGÍA Y TRAUMATOLOGÍA BUCO MAXILOFACIAL,"Complejo Hospitalario San José (Santiago, Inde...",Independencia,Desconocido,27.0,"[25, 30)","[20, 30)",1


# Graficar series de tiempo

In [13]:
# Generamos una función que toma como parámetro el nombre de la serie:
# 'Consultas_producidas', 'Consultas_inasistencia', 'Oferta_consultas', 'Consultas_solicitadas', 'Lista_espera_inicial'

def graficar_serie(serie):

  df = cubo.groupby(['Año','Centro']).agg({serie:'sum'}).reset_index()

  # Generar el gráfico
  fig = px.line(df,
                x="Año",
                y=serie,
                color="Centro",
                title=serie+'  por Periodo y Centro')
  fig.show()

# Ejecutamos la función

graficar_serie('Interconsultas')

In [14]:
# También podemos agregar un parámetro que permita filtrar por especialidad

def graficar_serie_especialidad(serie, especialidad):

  # generamos una variable para filtrar por especialidad
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  df = cubo[filtro_especialidad].groupby(['Año','Centro']).agg({serie:'sum'}).reset_index()


  # Generar el gráfico
  titulo = serie+'  por Año y Centro ('+especialidad+')'
  fig = px.line(df,
                x="Año",
                y=serie,
                color="Centro",
                title=titulo)
  fig.show()

graficar_serie_especialidad('Interconsultas', 'OBSTETRICIA')

# Análisis

* ¿Cuál fue la demanda de interconsultas de Gastroenterología y Coloproctología por centro entre 2019 y 2023?
* ¿Cuál fue la demanda de interconsultas de Pediatría el 2023?
* ¿Qué otra pregunta podría responder utilizando esta base de datos y funciones presentadas aquí?
